In [1]:
import os
import json
import glob
import pandas as pd
import numpy as np

In [2]:
PASS = 0
REJECT = -1

CATEGORY_D = ["BICYCLE", "CAR", "BUS", "ETC", "MOTORCYCLE", "PEDESTRIAN", "TRUCK"]
CATEGORY_S = ["MEDIAN_STRIP", "OVERPASS", "RAMPSECT", "ROAD_SIGN", "SOUND_BARRIER", "STREET_TREES", "TUNNEL"]
CATEGORY_ATYP = ["MEDIAN_STRIP", "OVERPASS", "RAMPSECT", "SOUND_BARRIER", "TUNNEL"]

COLUMNS = ["clip 이름", "프레임 번호", "객체 id", "반려사유"]
SUM_COL = ["clip 이름", "검수결과", "반려사유"]

FILE_NUM_ERROR, FRAME_ERROR, CATEGORY_ERROR, ATYP_ERROR, LOC_ERROR, PCD_ERROR, POLYGON_ERROR, DIM_ERROR, GROUP_ERROR, SCENCE_ERROR = list(range(1,11))

dic = {
    PASS: "통과",
    REJECT: "반려",
    FILE_NUM_ERROR: "파일 개수 오류",
    FRAME_ERROR: "빈 프레임 오류",
    CATEGORY_ERROR: "객체종류 오류",
    ATYP_ERROR: "비정형/정형 오류",
    LOC_ERROR: "박스 위치 오류",
    PCD_ERROR: "PCD 부족 오류",
    POLYGON_ERROR: "폴리곤 오류",
    DIM_ERROR: "크기 오류",
    GROUP_ERROR: "그룹화 오류",
    SCENCE_ERROR: "시나리오 오류"
}


In [3]:
print("세령이 등장!!!!!!!!!!!!!!!!!!!!!!!!!!")
print("검수도구__ver. 0.2")
print("단일 json 단위만 구현 중(기준 50% 완료) / 모듈화 예정 / multi json 검수 추가예정")

세령이 등장!!!!!!!!!!!!!!!!!!!!!!!!!!


In [530]:

class ErrorLogger:
    
    def __init__(self, clip_name):
        
        self.clip_name = clip_name
        self.is_reject = PASS
        self.err_type = []
        self.err_logging = pd.DataFrame(columns=COLUMNS)
    
    def set_clip_name(self, name):
        self.clip_name = name
    def set_reject(self, reject):
        self.is_reject = reject
    def add_err_type(self, err):
        self.err_type.append(err)
    def add_err_log(self, frame, object_id, err_type):
        row = [[self.get_clip_name(), frame, object_id, dic.get(err_type)]]
        l = pd.DataFrame(row, columns=COLUMNS)
        self.err_logging = pd.concat([self.get_err_log(), l], ignore_index=True)
        self.set_reject(REJECT)
        self.add_err_type(err_type)
        del l
    def add_pass_log(self):
        row = [[self.get_clip_name(), -1, -1, dic.get(PASS)]]
        l = pd.DataFrame(row, columns=COLUMNS)
        self.err_logging = pd.concat([self.get_err_log(), l], ignore_index=True)
        self.set_reject(PASS)
        self.add_err_type(PASS)
        del l
    
        
    def get_clip_name(self):
        return self.clip_name
    def get_reject(self):
        return self.is_reject
    def get_err_type(self):
        return set(self.err_type)
    def get_err_log(self):
        return self.err_logging
    def get_summary(self):
        root_clip = self.get_clip_name()
        res = dic.get(self.get_reject())
        
        err_type_list = list(self.get_err_type())
        err_type_list.sort()
        
        for i in range(len(err_type_list)):
            tmp = dic.get(err_type_list[i])
            err_type_list[i] = tmp
        err_type = ",".join(err_type_list)
        row = [[root_clip, res, err_type]]
        ret = pd.DataFrame(row, columns=SUM_COL)
        return ret
        

In [ ]:
"""
추가예정 조건식 - 단일 JSON

PCD_ERROR: "",
POLYGON_ERROR: "",
DIM_ERROR: "",
GROUP_ERROR: "",
SCENCE_ERROR: ""

에러로그 한글화

"""

In [531]:

def rej_single_res(base_path, clip):
    res = glob.glob(f'{base_path}/{clip}/*')
    
    err_logger = ErrorLogger(clip)
    
    if len(res)<100: 
        print(f"{clip} reject labeling: not enough json number")
        err_logger.add_err_log(-1, -1, FILE_NUM_ERROR)
        return err_logger
    
    for r in res:
        with open(r, "r") as f:
            data = json.load(f)
            fnum = data["frame_no"]
            annot = data["annotation"]
            
            if annot == []:
                print(f"{clip}: frame {fnum}: is empty")
                err_logger.add_err_log(fnum, -1, FRAME_ERROR)
                
            for obj in annot:
                id = obj["id"]
                cat = obj["category"]
                type = obj["obj_type"]
                is_atyp = obj["atypical_yn"]
                box_list = obj["3d_box"]
                cam_vis = obj["camera_visibility"]
                polygon = obj["2d_polygon"]
                if type==0:
                    if cat not in CATEGORY_D: 
                        print(f"{clip}: frame {fnum}: object {id}: wrong category")
                        err_logger.add_err_log(fnum, id, CATEGORY_ERROR)
                else:
                    if cat not in CATEGORY_S:
                        print(f"{clip}: frame {fnum}: object {id}: wrong category")
                        err_logger.add_err_log(fnum, id, CATEGORY_ERROR)
                        
                if (cat in CATEGORY_ATYP): 
                    if is_atyp=="n":
                        print(f"{clip}: frame {fnum}: object {id}: wrong atypical_yn")
                        err_logger.add_err_log(fnum, id, ATYP_ERROR)
                else: 
                    if is_atyp=="y":
                        print(f"{clip}: frame {fnum}: object {id}: wrong atypical_yn")
                        err_logger.add_err_log(fnum, id, ATYP_ERROR)
                for box in box_list:
                    loc = box["location"]
                    if loc[0] < 0:
                        print(f"{clip}: frame {fnum}: object {id}: wrong location")
                        err_logger.add_err_log(fnum, id, LOC_ERROR)    
                    pcd_cnt = box["lidar_point_count"] + box["radar_point_count"]    
                    # if pcd_cnt < 3:
                    #     print(f"{clip}: frame {fnum} object {id} lack of point cloud")
    if err_logger.get_reject() == PASS:
        err_logger.add_pass_log()
    
    return err_logger
    
    
def iterate_clip(base_path):
    clip_folders = [i for i in os.listdir(base_path) 
                    if 'Clip_' in i]
    
    sum_log_df = pd.DataFrame(columns=SUM_COL)
    full_log_df = pd.DataFrame(columns=COLUMNS)
    
    rejected_clip = []
    for clip in clip_folders:
        el = rej_single_res(base_path, clip)
        
        sum_log = el.get_summary()
        sum_log_df = pd.concat([sum_log_df, sum_log], ignore_index=True)
        
        full_log = el.get_err_log()
        full_log_df = pd.concat([full_log_df, full_log], ignore_index=True)
        if el.get_reject() == REJECT:
            rejected_clip.append(clip)
            
    return sum_log_df, full_log_df


def get_scene(clip):
    ## 클립명에서 목표 객체 (동적or주행환경) 추출
    
    pass



In [532]:
# base = r"C:\workspace\rejecter\가공완료 클립\extract_2022-08-02-18-18-53"
# clip = "Clip_00033"
# el = reject_res(base, clip)
# res = el.get_err_log()

base = r"C:\workspace\rejecter\가공완료 클립\extract_2022-08-02-18-18-53"
sum, full = iterate_clip(base)
sum.to_csv("./sum.csv")
full.to_csv("./full.csv")

Clip_00001: frame 2: object 6: wrong atypical_yn
Clip_00001: frame 3: object 6: wrong atypical_yn
Clip_00001: frame 4: object 6: wrong atypical_yn
Clip_00001: frame 5: object 6: wrong atypical_yn
Clip_00001: frame 6: object 6: wrong atypical_yn
Clip_00001: frame 7: object 6: wrong atypical_yn
Clip_00001: frame 8: object 6: wrong atypical_yn
Clip_00001: frame 9: object 6: wrong atypical_yn
Clip_00001: frame 10: object 6: wrong atypical_yn
Clip_00001: frame 11: object 6: wrong atypical_yn
Clip_00001: frame 12: object 6: wrong atypical_yn
Clip_00001: frame 13: object 6: wrong atypical_yn
Clip_00001: frame 14: object 6: wrong atypical_yn
Clip_00001: frame 15: object 6: wrong atypical_yn
Clip_00001: frame 16: object 6: wrong atypical_yn
Clip_00001: frame 17: object 6: wrong atypical_yn
Clip_00001: frame 18: object 6: wrong atypical_yn
Clip_00001: frame 19: object 6: wrong atypical_yn
Clip_00001: frame 20: object 6: wrong atypical_yn
Clip_00001: frame 21: object 6: wrong atypical_yn
Clip_000

In [533]:
# res.to_csv("./rejecter.csv")


In [534]:
# base_path = r'C:\workspace\rejecter\가공완료 클립\extract_2022-08-01-10-53-48'

# clip_folders = [i for i in os.listdir(base_path) 
#                 if len(i) == 10 and 'Clip_' in i]

# rejected_clip = []
# for clip in clip_folders:
#     ret, logging = rej_res(base_path, clip)
#     if ret != PASS:
#         rejected_clip.append(ret)
#         err_logging = set(logging)
#         for err in err_logging:
#             print(dic.get(err))
#         print("================")

# print("=========")
# print(rejected_clip)